# Channel Break Out Detection Alcista

In [1]:
import pandas as pd
import numpy as np
#import plotly.graph_objects as go
import yfinance as yf
import talib as ta
#import matplotlib.pyplot as plt
#import math
from datetime import date, timedelta
from scipy import stats
from scipy.signal import argrelextrema
#from bokeh.plotting import figure, show, column
from sklearn.linear_model import LinearRegression
import os
from backtesting import Backtest, Strategy

/usr/share/miniconda/envs/test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
df_h=pd.read_csv('data/dataxh.txt',sep='\t')

tickers = [
    'SPY',
    'META',
    'AAPL',
    'AMZN',
    'NFLX',
    'MRNA',
    'TSLA',
    'TNA',
    'GLD',
    'SLV',
    'USO',
    'BAC',
    'CVX',
    'XOM',
    'QQQ',
    'MSFT',
    'NVDA',
    'WMT',
    'BA',
    'DIS',
    'CAT',
    'IBM',
    'WFC',
    'PLTR',
    'AMD',
    'AVGO',
    'HOOD',
    'CRWV',
    'MSTR',
    'UNH',
    'GOOG',
    'APP',
    'UBER'
]

#    'SPY',
#    'META',
#    'AAPL',
#    'AMZN',
#    'NFLX',
#    'MRNA',
#    'TSLA',
#    'TNA',
#    'GLD',
#    'SLV',
#    'USO',
#    'BAC',
#    'CVX',
#    'XOM',
#    'QQQ'

In [3]:
df_h['datetime'] = pd.to_datetime(df_h['datetime'])
df = pd.DataFrame()
for ticker in tickers:
    company = df_h.query("companyName==@ticker").copy()
    company.sort_values(by=['datetime'])
    #company = yf.download(ticker, start = ini2_str, end = today_str, interval='60m')
    #company.columns = [company.columns[0][0], company.columns[1][0], company.columns[2][0], company.columns[3][0], company.columns[4][0]]
    #company.rename(columns={'Datetime':'Gmt time'}, inplace = True)
    #company['datetime'] = pd.to_datetime (company.index)
    #company['companyName'] = ticker
    company['SMA20'] = company['Close'].rolling(20).mean()
    company.dropna(inplace=False)
    company['SMA40'] = company['Close'].rolling(40).mean()
    company.dropna(inplace=False)
    company['SMA100'] = company['Close'].rolling(100).mean()
    company.dropna(inplace=False)
    company['SMA200'] = company['Close'].rolling(200).mean()
    company.dropna(inplace=False)
    df = pd.concat([df, company],ignore_index=True)

### Detect Pivots/Fractals

In [4]:
ord=15
#for ticker in tickers:
#print("====", ticker)
#dfpl = df[(df.companyName==ticker)] #df.query("companyName=='AAPL'")
max_idx = argrelextrema(df['Close'].values, np.greater, order=ord)[0]
min_idx = argrelextrema(df['Close'].values, np.less, order=ord)[0]
#print(max_idx)
#print(min_idx)
#df['pivotHigh'] = np.nan
#df['pivotLow'] = np.nan
#df['isPivot'] = np.nan
#df['isPivot'] = np.nan
# Aplicar el cálculo solo a los índices en la lista
df.loc[max_idx, 'pivotHigh'] = df['High']+1e-3
df.loc[min_idx, 'pivotLow'] = df['Low']-(1e-3)
df.loc[max_idx, 'isPivot'] = 1
df.loc[min_idx, 'isPivot'] = 2

##### Canales alcista, cuando PM40 esta por encima o cerca del precio

In [5]:
def rcb01(df):
    if ((df['isPivot']==1) & ((df['SMA40']>df['Close']) | (df['SMA40']>df['Open']) | (df['SMA20']>df['SMA40']))):
        return 1
    else:
        return 0
df['rcb01_01'] = df.apply(rcb01, axis=1)

#### Detect Price Channel

In [6]:
def collect_channel(candle, backevalTrend, trendL):
    #localdf = df[candle-backcandles-window:candle-window]
    localdf = dfpl.loc[candle-backevalTrend:candle] #tomar en cuenta el backcandles
    #localdf['isPivot'] = localdf.apply(lambda x: isPivot(x.name,window), axis=1)
    
    #highs = localdf.High.values
    #idxhighs = localdf.High.index
    #lows = localdf.Low.values
    #idxlows = localdf.Low.index

    #highs = localdf.loc[localdf.index.isin(trendH)].High.values
    #idxhighs = localdf.loc[localdf.index.isin(trendH)].High.index
    lows = localdf.loc[localdf.index.isin(trendL)].Low.values
    idxlows = localdf.loc[localdf.index.isin(trendL)].Low.index

    #highs = localdf.loc[localdf.index.isin([candle]) | (localdf['isPivot'] == 1)].High.values
    #idxhighs = localdf.loc[localdf.index.isin([candle]) | (localdf['isPivot'] == 1)].High.index
    #lows = localdf.loc[localdf.index.isin([candle]) | (localdf['isPivot'] == 2)].Low.values
    #idxlows = localdf.loc[localdf.index.isin([candle]) | (localdf['isPivot'] == 2)].Low.index

    #lows = localdf.loc[localdf.index.isin(trendL)].Low.values
    #idxlows = localdf.loc[localdf.index.isin(trendL)].Low.index

    #print("highs:",highs)
    #print("idxhighs:",idxhighs)
    #print("lows:",lows)
    #print("idxlows:",idxlows)
    #print ("tamanio:",localdf.shape[0])
    
    #if len(lows)>=2 and len(highs)>=2:
    if len(lows)>=2:
        sl_lows, interc_lows, r_value_l, _, _ = stats.linregress(idxlows,lows)
        #sl_highs, interc_highs, r_value_h, _, _ = stats.linregress(idxhighs,highs)    
        #return(sl_lows, interc_lows, sl_highs, interc_highs, r_value_l**2, r_value_h**2)
        return(sl_lows, interc_lows, r_value_l**2)
    else:
        return(0,0,0)
    

In [7]:
def backcandle(candle):
    
    trendH = []
    backeval = 0
    val = 0

    #print("candle:", candle)
    dfpl['PM20_PM40'] = dfpl['SMA20']-dfpl['SMA40']
    idinicial = dfpl.index[0]
    velas = candle - idinicial

    #print("idinicial:", idinicial)
    if (dfpl.loc[candle,"PM20_PM40"]>0):
        idx_negativo = dfpl.loc[:candle, 'PM20_PM40'][::-1].lt(0).idxmax()
        if (idx_negativo==candle):
            backeval = candle - velas
            
        else:
            backeval = idx_negativo - 4
        
    val = candle - backeval
    
    #print("backeval1:",backeval)
    
    if val>velas:
        backeval=candle-velas

    #print("backeval2:",backeval)
    if ((dfpl.loc[backeval,"PM20_PM40"]>0) & (val<velas)):
        idx_negativo = dfpl.loc[:backeval, 'PM20_PM40'][::-1].lt(0).idxmax()
        backeval = idx_negativo - 4
    #print("backeval3:",backeval)

    val = candle - backeval
    if val>velas:
        backeval=candle-velas
        
    #print("backeval4:",backeval)
    #ini = candle - backeval
    ini = backeval
    trendprev=dfpl.loc[ini:candle]
    trend = trendprev.reset_index(drop=False)

    idxlows = trend.index.to_numpy().reshape(-1,1) # Convertir X a una matriz 2D
    lows = trend.Low.values
    modelo1 = LinearRegression()
    modelo1.fit(idxlows, lows)
    # Obtener la predicción de la línea de regresión
    Y_pred = modelo1.predict(idxlows)
    # Calcular la distancia solo de los puntos que están POR ENCIMA de la línea
    trend['Distancia'] = Y_pred - trend["Low"]  # Diferencia entre el valor real y la predicción
    # Filtrar los puntos que están por debajo (donde Y > Y_pred, es decir, Distancia > 0)
    df_encima = trend[trend['Distancia'] > 0]
    puntos_mas_lejanos = df_encima.nlargest(2, 'Distancia')   #2 puntos mas lejanos
    trendL = puntos_mas_lejanos["index"].tolist()
    trendL.append(candle)   
    
    return trendL, backeval

### Detect Break Out

In [8]:
def trendChannel(row,candle, backeval, window, sl_highs, interc_lows):

    if (candle-backeval-window)<0:
        return np.nan

    #print("=== DATOS ===")
    #print("candle:",candle)
    #print("backcandles:",backcandles)
    #print("window:",window)
    #trendprevlow = (sl_lows*row['id'].shift(1).fillna(0) + interc_lows)
    #trendprevhigh = (sl_highs*row['id'].shift(1).fillna(0) + interc_highs)
    ini=candle-backeval
    fin=candle+window
    #x = np.array(range(candle-backcandles-window, candle+window))
    trendcurrlow = np.where(np.logical_or((row.index > fin),  (row.index <  ini)), np.nan, (sl_lows*row.index + interc_lows))
    #trendcurrhigh = (sl_highs*row['id'] + interc_highs)
    
    #print("candle:",candle,",backeval", backeval, ", ini:", ini, ", fin:", fin,   ", rowMin:", row.index[0],", rowMax:", row.index[-1])
    #trendcurrhigh = np.where(np.logical_or((row.index > fin),  (row.index <  ini)), np.nan, (sl_highs*row.index + interc_highs))
    return trendcurrlow

In [9]:
def isBreakOut(candleEval,backcandles, window, start, candle):
    if (candleEval-backcandles-window)<0:
        return 0

    if candleEval==start:
        prev_idx = candleEval
    else:
        prev_idx = candleEval-1
        
    prev_high = dfpl.loc[prev_idx].High
    prev_low = dfpl.loc[prev_idx].Low
    prev_close = dfpl.loc[prev_idx].Close
    
    curr_idx = candleEval
    curr_high = dfpl.loc[candleEval].High
    curr_low = dfpl.loc[candleEval].Low
    curr_close = dfpl.loc[candleEval].Close
    curr_open = dfpl.loc[candleEval].Open

    trend_prevlow=dfpl.loc[prev_idx].trendcurrlow
    #trend_prevhigh=dfpl.loc[prev_idx].trendcurrhigh

    trend_currlow=dfpl.loc[candleEval].trendcurrlow
    #trend_currhigh=dfpl.loc[candleEval].trendcurrhigh

    #codigo para tendencia LOW, nos importa breaks en tendencia HIGH
    if ( prev_high > trend_prevlow and
        prev_close < trend_prevlow and
        curr_open < trend_currlow and
        curr_close < trend_currlow and 
        candleEval>candle): #and r_sq_l > 0.9
        return 1
    
   # #if ( prev_low < trend_prevhigh and
   #     prev_close > trend_prevhigh and
   #     curr_open > trend_currhigh and
   #     curr_close > trend_currhigh and
   #     candleEval>candle): #and r_sq_h > 0.9
   #     return 2

   # elif (curr_open > trend_currhigh and
   #     curr_close > trend_currhigh and candleEval>candle):
   #     return 3
    
    else:
        return np.nan

In [10]:
def breakpointpos(x):
    if x['isBreakOutIni'] in (2,3):
        return x['Low']+3e-3
    elif x['isBreakOutIni']==1:
        return x['High']-3e-3
    else:
        return np.nan

###### Hallando casos de Ruptura del Canal Alcista

In [11]:
def revisionVelas(dfpl, backeval,window, candle):
    #Revision de Velas
    #Promedio del volumen de 50 velas antes del posible caso
    promVol = dfpl.loc[i-backeval:i]["Volume"].mean()
    #puede darse cualquiera de estas condiciones: 
    # - que la vela sea verde, que tenga volumen, este por encima de la linea trend y que tenga cuerpo (cierra por encima de su tercera parte)
    # - que la vela sea envolvente, alcista y que este por encima de la linea trend
    j=candle+1
    k=candle+window
    ind_vela=0
    
    #k no puede pasrse del rango
    maxindex = dfpl.index[-1]
    if (k>maxindex):
        k=maxindex
    
    #print("candle:",candle)
    #print("window:",window)
    #print("j:",j, ",k:",k)
    
    #Evaluacion de velas ALCISTAS
    dfpl["cdlengulfing"] = ta.CDLENGULFING(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlhammer"] = ta.CDLHAMMER(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlmorningstar"] = ta.CDLMORNINGSTAR(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlpiercing"] = ta.CDLPIERCING(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlclosingmarubozu"] = ta.CDLCLOSINGMARUBOZU(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlmarubozu"] = ta.CDLMARUBOZU(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdl3whitesoldiers"] = ta.CDL3WHITESOLDIERS(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlharami"] = ta.CDLHARAMI(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlharamicross"] = ta.CDLHARAMICROSS(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlinvertdhammer"] = ta.CDLINVERTEDHAMMER(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlladderbottom"] = ta.CDLLADDERBOTTOM(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])

    dfpl['Vela_Roja'] = dfpl['Close'] < dfpl['Open']
    # Calcular el nivel de la tercera parte
    #dfpl['Third_Level'] = dfpl['Low'] + (dfpl['High'] - dfpl['Low']) / 3
    dfpl['Third_Level'] = dfpl['Open'] + (dfpl['High'] - dfpl['Low']) / 3
    # Verificar si la vela cierra por encima de su tercera parte, vela con cuerpo
    dfpl['Above_Third'] = dfpl['Close'] > dfpl['Third_Level']

    while j<=k:            
        
        vol = dfpl.loc[j,"Volume"]
        above_Third = dfpl.loc[j,"Above_Third"]
        vela_Roja = dfpl.loc[j,"Vela_Roja"]
        lineTrend = dfpl.loc[j,'trendcurrlow']
        high = dfpl.loc[j,'High']
        close = dfpl.loc[j,'Close']

        cdlengulfing = dfpl.loc[j,"cdlengulfing"]
        cdlhammer = dfpl.loc[j,"cdlhammer"]
        cdlmorningstar = dfpl.loc[j,"cdlmorningstar"]
        cdlpiercing = dfpl.loc[j,"cdlpiercing"]
        cdlclosingmarubozu = dfpl.loc[j,"cdlclosingmarubozu"]
        cdlmarubozu = dfpl.loc[j,"cdlmarubozu"]
        cdl3whitesoldiers = dfpl.loc[j,"cdl3whitesoldiers"]
        cdlharami = dfpl.loc[j,"cdlharami"]
        cdlharamicross = dfpl.loc[j,"cdlharamicross"]
        cdlinvertdhammer = dfpl.loc[j,"cdlinvertdhammer"]
        cdlladderbottom = dfpl.loc[j,"cdlladderbottom"]

        #vela consecutiva alcista
        candle_up = (#dfpl.loc[j,"Vela_Verde"]==True 
                     #and dfpl.loc[j-1,"Vela_Verde"]==True 
                     dfpl.loc[j,"High"]> dfpl.loc[j-1,"High"]
                     and dfpl.loc[j-1,"High"]> dfpl.loc[j,"Low"]
                     and dfpl.loc[j,"Low"]> dfpl.loc[j-2,"High"]
                     and dfpl.loc[j-2,"High"]> dfpl.loc[j-1,"Low"]
                     and dfpl.loc[j-1,"Low"]> dfpl.loc[j-3,"High"]
                     and dfpl.loc[j-3,"High"]> dfpl.loc[j-2,"Low"]
                     and dfpl.loc[j-2,"Low"]> dfpl.loc[j-3,"Low"]
                     #and dfpl.loc[j,"Close"]> dfpl.loc[j-1,"High"] 
                     #and dfpl.loc[j-1,"Close"]>dfpl.loc[j-2,"High"]
                     #and dfpl.loc[j-1,"Close"]>dfpl.loc[j-3,"High"] 
                    )

        if (candle_up==True and close>lineTrend):
            ind_vela=ind_vela+1
        #vela sea verde, que tenga volumen, que tenga cuerpo y que este por encima de la linea trend
        #if (vela_Verde==True and promVol<=vol and above_Third==True and close>=lineTrend):
        #    ind_vela=ind_vela+1
        
        #vela sea envolvente, alcista y que este por encima de la linea trend
        #if (
        #    (cdlhammer>0 or
        #     cdlmorningstar>0 or
        #     cdlpiercing>0 or
        #     cdl3whitesoldiers>0 or
        #     cdlharami>0 or
        #     cdlharamicross>0 or
        #     cdlinvertdhammer>0 or
        #     cdlladderbottom>0
        #    )
        #    and promVol<=vol
        #    and close>=lineTrend):
        #    ind_vela=ind_vela+1

        #vela sea envolvente, alcista y que este por encima de la linea trend
        #if (
        #    (cdlclosingmarubozu>0 or
        #     cdlmarubozu>0 or
        #     cdlengulfing>0 or
        #     cdlhammer>0
        #    )
        #    #and promVol<=vol
        #    and close>=lineTrend):
        #    ind_vela=ind_vela+1 
            
        j=j+1
    
    return 1 #1 para mostrar todo ind_vela para que funcione el filtro

In [12]:
df_casos = pd.DataFrame()
caso = 0
ticker2 = ""
window=5
backeval=75
backevalTrend=0
dfpl = pd.DataFrame
trendH = []
trendL = []
cant = 0
for i, row in df.iterrows():
    #if (i==3560):
    if(df['rcb01_01'][i]==1): #posibles casos 
        print("===posible caso:",i, df['companyName'][i])
        cant = 0
        candle = i
        cnt = 0
        #valiniHigh = df.loc[candle,"High"]
        #valiniLow = df.loc[candle,"Low"]
        ticker = df['companyName'][i]
        #Reinicio de casos por company
        if ticker2 != ticker:
            ticker2 = ticker
            caso = 0
        dfpl = df[(df.companyName==ticker)].loc[i-backeval:i+backeval]       

        #print("rowMin:", dfpl.index[0],", rowMax:", dfpl.index[-1])
                    
        h=1
        l=1

        cant = cant +1
        cantHorasTrend=0
        #print("opcionH:",h,"opcionL:",l)
        trendL, backevalTrend = backcandle(candle)
        #print ("-->trendH:",trendH)
        #print ("-->backevalTrend:",backevalTrend)    
        
        cantHorasTrend = (dfpl.loc[candle,"datetime"] - dfpl.loc[backevalTrend,"datetime"]).total_seconds() / 3600

        if (cantHorasTrend>48): #Verificacion de caida por lo menos 2 dias
            backevalTrend2 = candle - backevalTrend
            #print ("backevalTrend2", backevalTrend2)
                        
            #revision de trend negativo indica que esta cayendo, solo me quedo con estos casos
            sl_lows, interc_lows, r_sq_l = collect_channel(candle, backevalTrend2, trendL)
                    
    
            #print("======:", candle)
            #print("===trendH===")
            #print(trendH)
            #print("===trendL===")
            #print(trendL)
            dfpl.loc[trendL, "trendL"] = 1
                                
            #print("sl_lows:", sl_lows, ", interc_lows:", interc_lows, ", r_sq_l:", r_sq_l)
            if (sl_lows>=0): #solo tendencias a la alza and ind_vela>0 sl_lows<0 and r_sq_h>=0.7  and sl_highs<=-0.1 and ind_vela>0
                
                #crear linea de tendencia high y low
                dfpl['trendcurrlow']  = trendChannel(dfpl,candle,backevalTrend2,window,sl_lows,interc_lows)
                
                ind_vela = revisionVelas(dfpl, backeval,window, candle)
                promVol = dfpl.loc[i-backeval:i]["Volume"].mean()
    
                #print("ind_vela:",ind_vela)
                if (ind_vela>0):
                    dfpl["bearishSlope"] = np.nan #pendiente bajista en vacio
                    dfpl.loc[i, "bearishSlope"] = 1 #pendiente bajista solo al punto evaluado            
                    dfpl["ind_posicion"] = 0
                    dfpl.loc[(dfpl.index < i), 'ind_posicion'] = -1
                    dfpl.loc[(dfpl.index > i), 'ind_posicion'] = 1
                    dfpl["promVol"] = promVol
                    dfpl["sl_lows"] = sl_lows
                    dfpl["r_sq_l"] = r_sq_l
            
                    start = dfpl.index[0]
                    dfpl["isBreakOut"] = [isBreakOut(candleEval, backevalTrend2, window, start, candle) for candleEval in dfpl.index]
                    # Solo me quedo con el primer BREAK OUT
                    cnt = dfpl.query("isBreakOut==1 and Close<Open").shape[0] 
                    id=0
                    id2=0
                    if cnt>0:
                        id = dfpl.query("isBreakOut==1 and Close<Open").index[0]                        
                        #dfpl['isBreakOutIni'] = np.where(id!=dfpl.index, np.nan, dfpl['isBreakOut'])                        
                        dfpl.loc[id,'isBreakOutIni'] = 1

                        k=0
                        while (k<=3):
                            dfpl['breakpointpos'] = dfpl.apply(lambda row: breakpointpos(row), axis=1)
                            cnt2 = dfpl.query("index>@id and isPivot==2").shape[0]
                            
                            if cnt2>0:
                                id2 = dfpl.query("index>@id and isPivot==2").index[0]
                                dfpl.loc[id2,'isBreakOutFinal'] = 1
                                k=4
                                print("h1")
                            else:
                                #revisar 25 velas mas
                                #print("paso8")
                                idfinal = dfpl.index[-1] 
                                idfinal2 = idfinal+25
                                if idfinal2 in df[df['companyName']==ticker].index:
                                    dfpl2 = df[(df.companyName==ticker)].loc[idfinal+1:idfinal2].copy()
                                    dfpl = pd.concat([dfpl, dfpl2],ignore_index=False)    
                                    print("h2")
                                else:                  
                                    dfpl['isBreakOutFinal'] = np.nan
                                    print("h3")

                                k=k+1
                                print("h4")
                                
                        
                    else:
                        dfpl['isBreakOutIni'] = np.nan                  
                        dfpl['breakpointpos'] = np.nan
            
                    if cnt>0:
                        #INSERT CASO
                        caso=caso+1                            
                        dfpl["caso"] = caso
                        print(i, df['companyName'][i])
                        if len(df_casos)<1:
                            df_casos = dfpl
                        else:
                            df_casos = pd.concat([df_casos, dfpl],ignore_index=False)


===posible caso: 88 SPY
===posible caso: 164 SPY


h1
164 SPY
===posible caso: 208 SPY
h1
208 SPY
===posible caso: 226 SPY


h2
h4
h1
226 SPY
===posible caso: 250 SPY
===posible caso: 276 SPY
===posible caso: 296 SPY
===posible caso: 388 SPY


h1
388 SPY
===posible caso: 450 SPY
===posible caso: 586 SPY


h1
586 SPY
===posible caso: 753 SPY
===posible caso: 830 SPY


...

===posible caso: 112914 UBER
===posible caso: 112979 UBER
===posible caso: 113024 UBER


h1
113024 UBER
===posible caso: 113057 UBER
===posible caso: 113141 UBER


h1
113141 UBER
===posible caso: 113271 UBER
h1
113271 UBER
===posible caso: 113302 UBER
===posible caso: 113356 UBER


===posible caso: 113391 UBER
===posible caso: 113421 UBER
h1
113421 UBER
===posible caso: 113456 UBER


h1
113456 UBER
===posible caso: 113496 UBER
===posible caso: 113557 UBER
h3
h4
h3
h4
h3
h4
h3
h4
113557 UBER


In [13]:
#descarga de informacion, estrategia: RUPTURA DEL CANAL BAJISTA
#path = r'C:\Users\carlo\OneDrive\Documentos\TRADER\traderapp\data\rcb_h.txt'
path='data/rca_h.txt'

# check whether the file exists
if os.path.exists(path):
    # delete the file
    os.remove(path)
else:
    # if the file does not exist.
    print("File does not exists. File needs to be created.")

#export DataFrame to text file
with open(path, 'a') as f:
    #df_string = appl_hor3.to_string(header=True, index=False, sep ='\t')
    df_casos.to_csv(path, header=True, index=None, sep='\t', mode='w')
    #f.write(df_string)

### BACKTESTING
### Usando paquete backtesting.py

In [14]:
df_casos2 = df_casos.query("isBreakOutIni==1 or isBreakOutFinal==1").copy()

In [15]:
# =============== BACKTESTING ================= #
# obteniendo dataframe backtesting
df_backTesting = pd.merge(df, df_casos2[['datetime','companyName', 'isBreakOutIni','isBreakOutFinal','caso']], on = ['companyName','datetime'], how='left')
# datos de trade se asigna una vela anterior debido a que backtesting.py toma entra como la siguiente vela
df_backTesting['isBreakOutIni'] = df_backTesting['isBreakOutIni'].shift(-1)
df_backTesting['isBreakOutFinal'] = df_backTesting['isBreakOutFinal'].shift(-1)
#modificamos indice con el compo datetime
df_backTesting.set_index('datetime', inplace=True)

In [16]:
#Strategia
class strategyRupturaCanal(Strategy):    
    def init(self):        
        self.breakout_entry = self.I(lambda x: x, self.data.isBreakOutIni) #Indica la señal entrada al backtesting
        self.breakout_exit = self.I(lambda x: x, self.data.isBreakOutFinal) #Indica la señal salida al backtesting

    def next(self):       
        #print(self.data)        
        if self.breakout_entry[-1] == 1:
            self.sell()  #COMPRA
        elif self.breakout_exit[-1] == 1 and self.position:
            self.position.close() #CIERRE

In [17]:
# Por cada TIcket
my_stats = {}
estadisticas = pd.DataFrame()          # lista para ticker
tradesprev = pd.DataFrame()     # lista para cada caso por ticker
for ticker in df_backTesting['companyName'].unique():
    ticker_data = df_backTesting[df_backTesting['companyName'] == ticker].copy() #Filtro por cada ticker
    bt = Backtest(ticker_data, strategyRupturaCanal, cash=10_000)
    my_stats = bt.run()
    listTrades = my_stats['_trades']
    tradesdf = pd.DataFrame(listTrades) #trades
    tradesdf['Ticker'] = ticker
    stats_filter={
        'Ticker':ticker,
        'EntryTime':my_stats['Start'],
        'ExitTime':my_stats['End'],
        'Return [%]': my_stats['Return [%]'],
        'CAGR [%]': my_stats['Return (Ann.) [%]'],
        'Sharpe Ratio': my_stats['Sharpe Ratio'],
        'Max. Drawdown [%]': my_stats['Max. Drawdown [%]'],
        'Win Rate [%]': my_stats['Win Rate [%]'],
        '# Trades': my_stats['# Trades'],
        'Expectancy [%]': my_stats['Expectancy [%]'],
        'Profit Factor': my_stats['Profit Factor'],
        'Duration': my_stats['Duration'],
        'Avg. Trade [%]':my_stats['Avg. Trade [%]'],
        'Max. Trade Duration':my_stats['Max. Trade Duration'],
        'Avg. Trade Duration':my_stats['Avg. Trade Duration']
    }
    stat=pd.DataFrame([stats_filter])
    if tradesprev.shape[0]==0:
        tradesprev = tradesdf.copy()
        estadisticas=stat.copy()
    else:
        tradesprev = pd.concat([tradesprev, tradesdf],ignore_index=True)
        estadisticas=pd.concat([estadisticas, stat],ignore_index=True)
    # Mostramos
    #bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")

Backtest.run:   0%|          | 0/3365 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/3221 [00:00<?, ?bar/s]

/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=452: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=1778: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=2929: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
                                                       

Backtest.run:   0%|          | 0/3411 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/3474 [00:00<?, ?bar/s]

...

/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=940: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=1101: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(


Backtest.run:   0%|          | 0/3302 [00:00<?, ?bar/s]

/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=1456: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=2428: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
                                                       

Backtest.run:   0%|          | 0/3360 [00:00<?, ?bar/s]

/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=872: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/usr/share/miniconda/envs/test/lib/python3.10/site-packages/backtesting/backtesting.py:954: UserWarning: time=1235: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
                                                       

In [18]:
ini = df_casos2[(df_casos2['isBreakOutIni']==1)].copy()
df_casos3_ini=ini.rename(columns={'Open': 'EntryPrice', 'datetime': 'EntryTime', 'companyName': 'Ticker'})[['EntryPrice','Ticker','caso','EntryTime']]
final = df_casos2[(df_casos2['isBreakOutFinal']==1)].copy()
df_casos3_final=final.rename(columns={'Open': 'ExitPrice', 'datetime': 'ExitTime', 'companyName': 'Ticker'})[['ExitPrice','Ticker','caso','ExitTime']]

In [19]:
df_casos3=df_casos3_ini.merge(df_casos3_final,on=['Ticker','caso'],how='outer')

In [20]:
trades = pd.merge(df_casos3, tradesprev, on = ['EntryTime','Ticker'], how='left')

In [21]:
def exitPrice(x):
    if pd.isna(x['ExitPrice_x']):
        return x['ExitPrice_y']
    else:
        return x['ExitPrice_x']
trades['ExitPrice'] = trades.apply(lambda row: exitPrice(row), axis=1)

In [22]:
trades = trades.rename(columns={'EntryPrice_x': 'EntryPrice'})
trades = trades.drop(['EntryPrice_y','ExitPrice_x', 'ExitPrice_y'], axis=1)

In [23]:
def exitTime(x):
    if pd.isna(x['ExitTime_x']):
        return x['ExitTime_y']
    else:
        return x['ExitTime_x']
trades['ExitTime'] = trades.apply(lambda row: exitTime(row), axis=1)

In [24]:
def duration(x):
    if pd.isna(x['Duration']):
        if pd.isna(x['ExitTime']):
            return pd.NaT
        else:
            return x['ExitTime']-x['EntryTime']
    else:
        return x['Duration']
trades['Duration'] = trades.apply(lambda row: duration(row), axis=1)

In [25]:
trades = trades.drop(['ExitTime_x', 'ExitTime_y'], axis=1)

In [26]:
trades=trades[['Ticker','EntryTime','ExitTime','EntryPrice','ExitPrice','Duration','Size','EntryBar','ExitBar','ReturnPct','PnL','caso']]

In [27]:
for item in estadisticas['Ticker'].unique():
    count = trades[trades["Ticker"] == item].shape[0]
    estadisticas.loc[estadisticas['Ticker'] == item, '# Trades'] = count

In [28]:
#EXPORTACION DE DATAFRAME
path = 'data/backtesting'

# Asegurarse de que el directorio existe
os.makedirs(path, exist_ok=True)

path_trades = os.path.join(path, 'trades_rca_h.csv')
path_estadisticas = os.path.join(path, 'estadisticas_rca_h.csv')

for file_path in [path_trades, path_estadisticas]:
    if os.path.exists(file_path):
        os.remove(file_path)
    else:
        print(f"File does not exist: {file_path}. It will be created.")

trades.to_csv(path_trades, header=True, index=False, sep='\t', mode='w')
estadisticas.to_csv(path_estadisticas, header=True, index=False, sep='\t', mode='w')
